In [61]:
from openai import OpenAI
from jobsearch.database import fetch_data_from_postgresql
from jobsearch.params import OPENAI_API_KEY
from tqdm import tqdm
from jsonschema import validate

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

import numpy as np
import pandas as pd
import json

# pd.options.display.max_colwidth = 100
# pd.options.display.max_rows = 20

In [62]:
# Load new ft data
local_ft_data = pd.read_csv('../../data/fine-tuning_data/salary_ft_data_2.csv')

# open prompt instructions
with open("../../data/prompts/salary_extraction_prompt_instructions_only.txt", "r") as f:
    instructions = f.read()


### PROCESS DATA

In [107]:
# pd.options.display.max_colwidth = 100

# # [json.loads(salary) for salary in local_ft_data['salary'] ]
# # json.loads(local_ft_data['salary'][0])['salaire'][0]

# # local_ft_data['salary'] = local_ft_data['salary'].apply(lambda x :json.loads(x))
# # [salary for salary in local_ft_data.salary if salary['salaire']]

# non_null_outputs = [(id, sal) for id, sal in enumerate(local_ft_data.salary) if sal['salaire'][0]['montant_min'] or sal['salaire'][0]['montant_max']]

# duplicate_record_ids = set([123, 231])
# non_null_ids = [id for id, sal in non_null_outputs]
# ids = list(set(non_null_ids) - duplicate_record_ids)

# non_null_df = local_ft_data.loc[ids]


### CONVERT FT DATA TO ALPACA FORMAT

In [134]:
with open("../../data/fine-tuning_data/salary_ft_data.jsonl", "a") as f:
    for row in non_null_df.iterrows():
        
        salary_dict = row[1]['salary']
        salary_json_str = json.dumps(salary_dict)
        
        description = row[1]['description']
        
        # Create a dictionary
        formatted_dict = {'instruction': instructions, 'input': description, 'output': salary_json_str}
        
        json_str = json.dumps(formatted_dict, ensure_ascii=False) + '\n'
        
        f.write(json_str)
        
        # print(json_str)


### PUSH DATASET

In [145]:
all_ft_data = pd.read_json('../../data/fine-tuning_data/salary_ft_data.jsonl', lines=True)

# read and split
train_df, test_df = train_test_split(all_ft_data, test_size=0.15, random_state=42, shuffle=True)

# push to hub
train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)

ds = DatasetDict()
ds["train"] = train_df
ds["test"] = test_df

dataset_name = "axel-rda/salary_extraction_ft_dataset"
ds.push_to_hub(dataset_name, branch="main", private=True)

/home/axelus/anaconda3/envs/ggjobsearch/lib/python3.11/site-packages/datasets/dataset_dict.py:1681: FutureWarning: 'branch' was deprecated in favor of 'revision' in version 2.15.0 and will be removed in 3.0.0.
You can remove this warning by passing 'revision=main' instead.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/489 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/axel-rda/salary_extraction_ft_dataset/commit/95d9f01d40f5ff79af790ea2471c750fd5194fe5', commit_message='Upload dataset', commit_description='', oid='95d9f01d40f5ff79af790ea2471c750fd5194fe5', pr_url=None, pr_revision=None, pr_num=None)